In [12]:
import pandas as pd
import numpy as np
from scipy.interpolate import interp1d

# URL der CSV-Datei auf GitHub
url = 'https://raw.githubusercontent.com/SilasBereit/turbomachinery-pumpenanalyse/main/volume_flow_data.csv'

# Lade die CSV-Datei mit Semikolon als Trennzeichen und ohne unnötige Spalten
df = pd.read_csv(url, sep=';', usecols=[0, 1])  # Wir verwenden nur die ersten zwei Spalten

# Manuelle Benennung der Spalten
df.columns = ['timestamp', 'flow_rate_m3h']

# Datentypen umwandeln
df['timestamp'] = pd.to_datetime(df['timestamp'], format='%d.%m.%y %H:%M')

# m³/h → m³/s
df['flow_rate_m3s'] = df['flow_rate_m3h'] / 3600

# Zeitdifferenz berechnen
df['time_delta_s'] = df['timestamp'].diff().dt.total_seconds()
df['time_delta_s'] = df['time_delta_s'].fillna(0)  # Keine Warnung mehr

# Kennlinie für Impeller Ø264 mm (geschätzt aus Diagramm)
flow_m3h = np.array([110, 240, 300, 350, 425, 495, 555])
head_m = np.array([23, 22, 21, 20, 18, 16, 14.3])

head_interpolator = interp1d(flow_m3h, head_m, bounds_error=False, fill_value="extrapolate")
df['head_m'] = head_interpolator(df['flow_rate_m3h'])

# Konstanten
rho = 1000       # Dichte [kg/m³]
g = 9.81         # Erdbeschleunigung [m/s²]

# Hydraulische Leistung
df['P_hydraulic'] = rho * g * df['flow_rate_m3s'] * df['head_m']  # [Watt]

# Berechnung der zugeführten Leistung (Input Power)
# Hier nehmen wir eine Annahme für den Gesamtwirkungsgrad der Pumpe
df['P_input'] = df['P_hydraulic'] / 0.7  # Angenommener Wirkungsgrad von 70% (0.7)

# Energie
df['E_hydraulic'] = df['P_hydraulic'] * df['time_delta_s']
df['E_input'] = df['P_input'] * df['time_delta_s']

# Gesamtenergie in kWh
total_energy_joule = df['E_input'].sum()
total_energy_kWh = total_energy_joule / 3.6e6

# Genutzte Energie in kWh (hydraulische Arbeit)
total_hydraulic_energy_joule = df['E_hydraulic'].sum()
total_hydraulic_energy_kWh = total_hydraulic_energy_joule / 3.6e6

# Pumpen-Effizienz berechnen (Verhältnis von genutzter Energie zu zugeführter Energie)
pump_efficiency = total_hydraulic_energy_kWh / total_energy_kWh * 100

# Ausgabe der Ergebnisse
print(f"Gesamte zugeführte Energie: {total_energy_kWh:.2f} kWh")
print(f"Gesamte genutzte Energie (hydraulische Arbeit): {total_hydraulic_energy_kWh:.2f} kWh")
print(f"Pumpen-Effizienz: {pump_efficiency:.2f}%")


Gesamte zugeführte Energie: 85.35 kWh
Gesamte genutzte Energie (hydraulische Arbeit): 59.75 kWh
Pumpen-Effizienz: 70.00%
